# Hola Mundo: Desde el fichero a la pantalla

## ¿Quién soy yo?

* Jesús Espino 

  * Twitter: @jespinog
  * Github: http://github.com/jespino
  * Email: jespinog@gmail.com
  
* Desarrollador en:

![](logos.png)

## ¿De qué vamos a hablar?
* Compiladores
* Intérpretes
* Python
* Hola Mundo

## ¿Qué es un compilador?
* Convierte un código fuente en código ejecutable por una máquina.
* Normalmente se divide en varias fases (análisis léxico, análisis sintáctico, construcción de árbol abstracto, generación de código máquina)

## ¿Qué es un interprete?
* Lee un código fuente, y lo interpreta directamente.
* Va leyendo y ejecutando el código tal cual.

## ¿Qué es python?
* ¿Es interpretado?
* ¿Es compilado?

<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />

## Ambas cosas

* Python compila el código a ByteCode
* El ByteCode es un código que interpreta la "Máquina Virtual" de python.

## Y ¿cómo lo hace?
![](compiler-graph.png)

##  Un paseo por el código de Python 

* https://github.com/python/cpython/tree/3.6/

## Hola Mundo: Del fichero a la pantalla

In [ ]:
source_code = open("holamundo.py", "r").read()
print(source_code)

In [ ]:
import holamundo
holamundo.holamundo()

## El resultado del compilador

* El compilador genera ByteCode, veamos como.

## Está usted aquí
![](compiler-graph-1.png)

In [ ]:
c = compile(source_code, filename="holamundo.py", mode="exec")

In [ ]:
import dis
dis.dis(c)

In [ ]:
dis.dis(holamundo.holamundo)

## ¿Pero qué demonios significa esto?

 * https://github.com/python/cpython/blob/3.6/Include/opcode.h
 * https://github.com/python/cpython/blob/3.6/Python/ceval.c#L1220

## ¿En serio? ¿Código fuente?

* Vaaaale... si lo prefieres tambien hay documentación:
  * https://docs.python.org/3.6/library/dis.html#python-bytecode-instructions

###  Entonces...

Esto es lo que interpreta la maquina virtual de python, ¿no?, Pues no.

In [ ]:
print(type(c))
print(c.co_code)
print([b for b in bytes(c.co_code)])
print(c.co_code.hex())

In [ ]:
print(type(holamundo.holamundo))
print(type(holamundo.holamundo.__code__))
print(holamundo.holamundo.__code__.co_code)
print([b for b in bytes(holamundo.holamundo.__code__.co_code)])
print(holamundo.holamundo.__code__.co_code.hex())

## Y ¿Como lo ejecuto?

* Usando `exec` o `eval`
* `exec` ejecuta un bloque de código y no devuelve resultado
* `eval` ejecuta una expresion y devuelve resultado

In [ ]:
exec(c)
holamundo()

## Pero ¿cómo llego hasta aquí?

* El compilado se divide principalmente en 6 fases:
    * Tokenizado.
    * Construcción del arbol sintáctico (parseado).
    * Construcción del arbol sintáctico abstracto (AST).
    * Construcción de la tabla de símbolos.
    * Traducción a ByteCode.
    * Optimización del ByteCode

## El primer paso: Tokenizado

El primer paso es el tokenizado del código, que divide y etiqueta cada uno de los elementos de nuestro código para luego ser procesados.

## Está usted aquí
![](compiler-graph-2.png)

In [ ]:
import tokenize

tokens = tokenize.tokenize(open('holamundo.py', "rb").readline)
print("\n".join([str(token) for token in tokens]))

### ¿Dónde ocurre la magia?

* https://github.com/python/cpython/blob/3.6/Parser/tokenizer.c#L1363

## Segundo paso: Análisis sintáctico
Tras el tokenizado se realiza el análisis sintáctico, que organiza nuestros tokens en un Syntax-Tree

## Está usted aquí
![](compiler-graph-3.png)

In [ ]:
import parser
st = parser.suite(source_code)
st.totuple()

Podemos verlo un poco mejor si traducimos los IDS a Tokens y Símbolos

In [ ]:
import stpp
stpp.stpp(st)

In [ ]:
mod = st.compile()
exec(mod)
holamundo()

### ¿Dónde ocurre la magia?

* pgen genera el código necesario para la gramática:
  * https://github.com/python/cpython/blob/3.6/Grammar/Grammar
* https://github.com/python/cpython/blob/3.6/Parser/parsetok.c#L183

## Tercer paso: Análisis semántico
Tras la construcción del análisis sintáctico se hace el análisis semántico y se construye un Abstract-Syntax-Tree

## Está usted aquí
![](compiler-graph-4.png)

In [ ]:
import ast
x = ast.parse(source_code)
print(ast.dump(x))

In [ ]:
import astpp
print(astpp.dump(ast.parse(source_code)))

### ¿Donde ocurre la magia?

* asdl_c.py genera el codigo necesario para la generación del AST:
  * https://github.com/python/cpython/blob/3.6/Parser/Python.asdl
* https://github.com/python/cpython/blob/3.6/Pyjthon/ast.c#L761

## Cuarto paso: Tabla de símbolos

Una vez construido el AST necesitamos la tabla de símbolos

## Está usted aquí
![](compiler-graph-5.png)

In [ ]:
import symtable
import symtablepp
table = symtable.symtable(source_code, filename="holamundo.py", compile_type="exec")
symtablepp.symtablepp(table)

## Quinto y sexto paso: Compilación y Optimización

Una vez tengo el AST y la tabla de símbolos, ya estoy listo para compilar el código.

* https://github.com/python/cpython/blob/3.6/Python/compile.c
* https://github.com/python/cpython/blob/3.6/Python/peephole.c#L425

## Está usted aquí
![](compiler-graph-6.png)

## Y aquí
![](compiler-graph-1.png)

## Resumen
![](compiler-graph.png)

# Dudas